MAA_MACHINE LEARNING TRANSLATION.

This system demonstrates a rule-based machine translation pipeline that maps English words to their Maasai equivalents using a custom-built bilingual dictionary. The translator includes a reverse translation feature (Maasai back to English) to assess accuracy and consistency, forming a foundational model for low-resource language translation."


The MAA MAchine learning methodology is a lexical-based translation system for the Maasai community. It begins with compiling and cleaning a custom dictionary. The system performs a simple word-for-word translation, handling unknown words by leaving them as-is. To evaluate its effectiveness, you use a combination of metrics:


BLEU Score (Accuracy): This score measures how many words in your translated text overlap with a professional human translation. A high score means your model is producing the right words.

Perplexity (Fluency): This score tells you how natural and predictable your translated sentences are. A low score indicates that your translated text is fluent and easy to read for a native speaker.

Coherence (Meaning): This score assesses whether the overall meaning and intent of the original message have been preserved in the translation. A high score proves that your model is effectively conveying the original message.

In [ ]:
import pandas as pd
import re
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from io import StringIO

# The data for the Maasai dictionary is now embedded in the code.
dict_data = """English,Maa (Maasai)
IEBC,IEBC
is,is
conducting,conducting
a,a
campaign,entoki sidai
to,to
increase,increase
voter,voter
registration,enkata esileto
in,in
all,all
counties,olchani lemeita
EACC,EACC
encourages,nemeishori enkurma
citizens,iltunganak
report,enkining’ori
corruption,enkarna
through,through
county,enkata
forums,ilinkopitoki
KNCHR,KNCHR
carries,nemepesho
out,out
awareness,entoki sidai
educate,esoma
the,the
public,iltunganak
about,about
constitutional,engai
rights,entoki leengai
NCIC,NCIC
brings,nemeishori empiron
peace,enyor sidai
initiatives,duo
prevent,empukunyik
election,enkarna
violence,esikirie
Council,Olmarei
of,le
Governors,Governors
oversees,nemeitoliki ilinkopitoki
forums,ilinkopitoki
planned,planned
services,services
National,Kenya
Treasury,Enkata oleng’amua
launches,nemeitat
training,entoki esoma
on,on
budget,enkatitin ena budget
writing,esidai nabo
promote,promote
transparency,uwazi
Youth,Ajenda
Agenda,le narua
plan,empiron
participation,enkarna
governance,enkata
NGEC,NGEC
promotes,nemeishori
public,iltunganak
dialogues,ilopito
gender,ishiligi nkiyena
equality,entoki le ishiligi nkiyena
advocates,nemeitoliki enkurma
for,duo
equal,entoki sidai
representation,enkatitin le ishiligi nkiyena
women,ishiligi nkiyena
elections,enkarna
Constitution,Taasisi le Katiba
Institute,Taasisi
runs,nemepesho
civic,entoki sidai
education,esoma
campaign,enkarna
on,on
rights,entoki leengai
IEBC,IEBC
digital,e-digital
platforms,enkiyieu
educate,esoma
new,enkarna
voters,iltunganak
campaigns,enkarna
promote,promote
anonymous,enkining’ori
reporting,enkining’ori
of,le
corruption,enkarna
cases,inkidung’u enkarna
citizens,iltunganak
inclusion,ilchan oleng’en
marginalized,ilchan lele enkop
communities,ilchan
peacebuilding,enyor sidai
cultural,entoki le olmarei
community,olmarei
events,empukunyik
how,jinsi
budgets,enkatitin ena budget
work,work
treasury,Enkata oleng’amua
sensitization,entoki sidai
programs,empiron
taxation,ena ushuru
compliance,enkerai
equips,nemeishori
young,narua
leaders,iltunganak narua
with,with
civic,entoki sidai
skills,enkatitin le entoki sidai
advocates,nemeitoliki enkurma
representation,enkatitin le ishiligi nkiyena
women,ishiligi nkiyena
in,in
elections,enkarna
public,iltunganak
participation,enkanashe
roles,enkanashe
organizes,nemeitoliki
forums,ilinkopitoki
electoral,enkarna
integrity,entoki
readiness,utayari
forms,nemeitodol
local,oleng’oni le sidai
watchdog,ileng’oni le sidai
groups,ileng’oni
fight,emunyunye
bribery,enkarna
starts,nemeitat
clubs,ilopi le nkera
teach,esoma
students,iltunganak le nkera
their,their
constitutional,engai
rights,entoki leengai
uses,nemeisho
peace,enyor sidai
caravans,empukunyik
spread,empukunyik
unity,enkarna
before,enkarna
elections,enkarna
launches,nemeitat
Reform,enkiyieu enkishui
Awareness,entoki sidai
Program,empiron
interactive,inkishui
tools,inkishui
explain,naibor
budgets,enkatitin ena budget
introduces,nemeisho
hackathons,ilopito
design,design
youth,narua
policies,enkatitin le narua
supports,nemeisho
women,ishiligi nkiyena
via,via
political,politika
mentorship,entoki e sidai
programs,empiron
produces,nemeisho ilopito
podcasts,ilopito
spread,spread
awareness,entoki leengai
partners,nemeisho enkiyieu
media,olasigirori
voter,esileto
content,content
brings,nemeisho
multiple,ilchan ileng’oni
channels,ilchan ileng’oni
for,duo
anonymous,enkining’ori
reporting,enkining’ori
promotes,nemeishori
civic,entoki sidai
education,esoma
people,iltunganak
with,with
disabilities,entoki olopolosho
forms,nemeitat
new,enkarna
peace,enyor sidai
committees,ilchani le sidai
address,address
local,oleng’oni
conflicts,manyata
tests,nemeisho
local,oleng’oni
hubs,ilchani le nkera
volunteer,volunteer
learning,esoma
responds,nemeisho iramat
to,duo
citizen,iltunganak
questions,ramat
on,on
budget,enkatitin le budget
allocations,enkatitin le budget
uses,nemeisho
tiktok,TikTok
teach,esoma
civic,entoki sidai
responsibility,entoki sidai
advocates,nemeisho empiron
for,duo
gender-equal,ishiligi nkiyena
budgeting,enkatitin le enkata
at,at
national,enkata
and,duo
ward,enche
levels,enche
provides,nemeisho
booklets,intitoi
on,on
rights,entoki leengai
and,duo
duties,enkurma
uses,nemeisho
boda,boda
boda,boda
networks,enkiyieu
spread,spread
voter,esileto
registration,entoki esileto
info,entoki esileto
works,nemeisho
with,with
civil,civil
groups,inkera e sidai
enforce,enforce
anti-corruption,enkarna esileto
rules,enkurma
launches,nemeitat
legal,enkai
help,empasurur
desks,empasurur
for,duo
marginalized,ilchan lele enkop
communities,ilchan
includes,nemeisho
peace,enyor sidai
studies,esoma
in,in
school,enkera
programs,empiron
runs,nemeisho ilasigirori
billboards,ilasigirori
raise,raise
awareness,esomoni
creates,nemeitat
programs,empiron
educate,esoma
citizens,iltunganak
on,on
public,enkata oleng
funds,enkata oleng
teaches,nemepesho
youth,narua
leaders,iltunganak narua
governance,enkata
principles,entoki
organizes,nemeitoliki
county-level,enkiyieu le county
events,empukunyik
gender,ishiligi nkiyena
unity,entoki le ishiligi nkiyena
introduces,nemeitat
e-learning,enkiyieu e-learning
for,duo
civic,entoki sidai
knowledge,entoki leengai
educates,nemepesho
public,iltunganak
new,enkarna
electoral,enkarna
reforms,enkarna sidai
forms,nemeitat
school,enkera
clubs,ilopi le nkera
promote,promote
anti-corruption,enkarna
ethics,entoki sidai
oversees,nemeitoliki
county,enkata
dialogues,enkopitoki
on,on
rights,entoki leengai
and,duo
equality,entoki sidai
uses,nemeisho
media,olasigirori
discourage,discourage
hate,hotuba
speech,hotuba
during,during
elections,enkarna
creates,nemeisho
podcast,ilasigirori
series,empiron le inkera
on,on
volunteer,volunteer
issues,issues
organizes,nemeitoliki
webinars,enkiyieu e-webinar
on,on
public,enkata oleng
finance,enkata oleng
accountability,entoki sidai
trains,nemepesho
youth,narua
use,enkiyieu
tech,enkiyieu e-technology
for,duo
civic,entoki sidai
engagement,entoki sidai
lawmakers,iltunganak le sheria
on,on
gender-sensitive,ishiligi nkiyena
laws,entoki sidai
creates,nemeisho
illustrations,ilasigirori
to,duo
aid,aid
civic,entoki sidai
education,esoma
launches,nemeitat
voter,esileto
registration,entoki esileto
plans,empiron
diaspora,esiligi e nenkop
introduces,nemeisho
mobile,e-mobile
app,enkore
for,duo
easy,e sidai
corruption,enkarna
reporting,enkining’ori
trains,nemepesho
partners,inkera e sidai
help,help
communities,ilchan
on,on
legal,entoki leengai
rights,entoki leengai
partners,nemeisho enkiyieu
with,with
religious,inkera e sidai
groups,inkera e sidai
promote,promote
peace,enyor sidai
open,nemeisho
civic,entoki sidai
education,enkopitoki
booths,intulie
in,in
rural,neituashu
areas,neituashu
provides,nemeisho
online,enkiyieu e-online
tools,inkishui
track,track
public,enkata oleng
funds,enkata oleng
advises,nemeisho
youth,narua
on,on
governance,enkata
and,duo
civic,entoki sidai
duty,entoki sidai
educates,nemepesho
public,iltunganak
on,on
GBV,GBV
reporting,entoki e sidai
and,duo
legal,entoki e sidai
redress,sheria
provides,nemeisho
documents,intitoi
on,on
citizen,iltunganak
rights,entoki leengai
collaborates,nemeisho enkiyieu
with,with
universities,e-university
on,on
voter,esileto
education,esoma
research,esoma
trains,nemepesho
media,olasigirori
professionals,olasigirori
on,on
corruption,enkarna
reporting,enkining’ori
offers,nemeisho
free,e sidai
legal,sheria
aid,entoki le sheria
during,during
public,enkopitoki
forums,enkopitoki
trains,nemepesho
local,oleng’oni
mediators,ileng’oni
in,in
conflict,emanyata
resolution,emanyata
provide,nemeisho
e-learning,e-learning
content,entoki e sidai
on,on
devolution,devolushon
trains,nemepesho
citizens,iltunganak
on,on
using,using
open,open
budget,entoki le budget
data,entoki le budget
challenges,nemeisho enkarna e sidai
youth,narua
create,create
civic,entoki sidai
tech,technology
solutions,solutions
campaigns,enkarna
for,duo
women’s,ishiligi nkiyena
representation,entoki sidai
in,in
county,enkata
assemblies,enkata
improves,nemeishori empiron
its,its
civic,entoki sidai
education,esoma
programs,empiron
uses,nemeisho
social,social
media,media
influencers,influencers
reach,esoma
young,narua
voters,iltunganak
supports,nemeisho
local,neituashu
offices,enkopitoki
preventing,preventing
misuse,entoki sidai
of,le
public,enkata oleng
funds,enkata oleng
uses,nemeisho
theatre,oltoyi
teach,esoma
human,iltunganak
rights,entoki leengai
principles,entoki leengai
sends,nemeisho
peace,enyor sidai
ambassadors,ileng’oni le sidai
to,duo
conflict-prone,esikirie
counties,enkata
uses,nemeisho
podcasts,podcast
raise,raise
public,enkata oleng
awareness,entoki sidai
leads,nemepesho
budget,entoki le budget
learning,esoma
for,duo
youth,narua
audiences,audiences
empowers,nemeisho
counties,enkata
and,duo
active,active
youth,narua
councils,halmashauri
organizes,nemeitoliki
national,Kenya
meetings,enkopitoki
on,on
inclusive,entoki e sidai
governance,enkata
creates,nemeisho
interactive,enkisui e sidai
tools,inkishui
for,duo
learning,learning
recruits,nemeisho
youth,narua
volunteers,iltunganak narua
for,duo
civic,entoki sidai
outreach,outreach
introduces,nemeisho enkiyieu
awards,tuzo
for,duo
reporting,enkining’ori
corruption,enkarna
airs,nemeisho
legal,entoki le sheria
advice,advice
and,duo
civic,entoki sidai
topics,topics
on,on
radio,e-radio
uses,nemeisho
art,sanaa
promote,promote
national,Kenya
unity,entoki sidai
messaging,entoki sidai
partners,nemeisho enkiyieu
with,with
journalists,ilasigirori
for,duo
devolution,devolushon
news,news
launches,nemeitat
AI-powered,AI
chatbot,chatbot
for,duo
budget,entoki le budget
questions,ramat
organizes,nemeitoliki
marathons,marathon
for,duo
voter,esileto
awareness,entoki sidai
creates,nemeisho enkiyieu
social,social
media,media
campaigns,empiron
on,on
gender,ishiligi nkiyena
equality,entoki le ishiligi nkiyena
counters,nemeisho
disinformation,enkarna
with,with
facts,enkarna
develops,nemeisho
interactive,enkidong’oi e sidai
games,enkidong’oi
to,duo
teach,esoma
election,enkarna
processes,processes
takes,nemeisho
anti-corruption,enkarna esileto
message,enkarna
to,duo
all,oleng
counties,enkata
launches,nemeitat
online,e-online
platforms,enkiyieu
for,duo
human,iltunganak
rights,entoki leengai
education,esoma
creates,nemeitat
online,e-online
spaces,enkiyieu
for,duo
conflict,emanyata
resolution,emanyata
dialogues,dialogues
rewards,nemeisho
best,sidai
counties,enkata
in,in
civic,enkopitoki
education,esoma
trains,nemepesho
journalists,ilasigirori
on,on
analyzing,analyzing
budget,entoki le budget
data,data
simulates,nemeisho
legal,entoki le sheria
sessions,sessions
for,duo
youth,narua
learning,esoma
monitors,nemeitoliki
gender,ishiligi nkiyena
equity,entoki sidai
progress,progress
in,in
targeted,targeted
sectors,sectors
funds,nemeisho
civic,enkisui e sidai
tech,technology
innovation,innovation
events,empukunyik
uses,nemeishori
door-to-door,door-to-door
campaigns,empiron
to,duo
reach,esoma
remote,neituashu
voters,iltunganak
Discussing,Entoki e sidai
citizen,iltunganak
freedom,freedom
in,duo
digital,e-digital
space,space
app,enkore
gives,nemeisho
learning,esoma
and,duo
reporting,enkining’ori
tips,tips
voter,esileto
education,esoma
for,duo
marginalized,ilchan lele enkop
groups,ilchan lele enkop
Illustrated,Intitoi e sidai
stories,intitoi
on,on
civic,enkopitoki
participation,entoki
and,duo
women,ishiligi nkiyena
Grassroots,entoki e sidai
trivia,Trivia
participation,entoki
on,on
Kenyan,Kenya
laws,sheria
Legal,sheria
text,intitoi
accessible,neitobir
to,duo
students,enkera
and,duo
the,the
public,iltunganak
Behavior,Enkisia
reflected,neitobir
in,duo
civic,enkopitoki
skits,enkisui
and,duo
tools,inkishui
Workshops,Intoki e sidai
held,nemetodol
in,duo
six,enene
high-tension,esikirie
counties,enkata
Depicts,Nemeisho
history,entoki e sidai
of,le
citizen,iltunganak
freedom,freedom
in,duo
Kenya,Kenya
Youth,narua
collaboration,enkiyieu
highlighted,nemeisho
after,after
3,duo
viral,viral
posts,posts
Focus,Entoki sidai
on,duo
grassroots,ilchan le neituashu
advocacy,entoki
strategies,entoki
Teams,Ileng’oni
compete,neitodol
on,duo
public,enkata oleng
accountability,entoki leengai
knowledge,entoki leengai
Targeted,enkarna
message,enkarna
sent,neishori
before,duo
county,enkata
forums,ilinkopitoki
Civic,enkopitoki
booths,intulie
set,nemeitodol
up,up
in,duo
Kakuma,Kakuma
and,duo
Dadaab,Dadaab
PDF,PDF
includes,nemeisho
legal,sheria
guidelines,intitoi le sheria
Murals,Inkidong’oi
and,duo
civic,enkopitoki
clubs,ilopi le enkopitoki
with,duo
local,neituashu
artists,ilopito
Event,Empukunyik
includes,nemeisho
civic,enkopitoki
talks,enkopitoki
and,duo
games,inkidong’oi
Social,social
media,media
training,empiron
for,duo
content,content
creators,ilasigirori
Series,Enkarna
follows,nemeisho
student,enkera
voter,esileto
group,group
Civic,enkopitoki
education,enkopitoki
section,section
on,on
financial,enkata le esidai
responsibility,entoki enkata le esidai
Statistics,Intoki e sidai
inform,nemeisho
county,enkata
sensitization,entoki sidai
plans,empiron
Student,nkera
discussions,inkopitoki
on,on
youth,narua
and,duo
civic,enkopitoki
power,entoki leengai
Participants,Iltunganak
solve,nemeisho
civic,enkopitoki
puzzles,enkidong’oi
Workshops,Intoki e sidai
for,duo
certified,certified
legal,sheria
advocates,iltunganak
New,enkarna
code,enkarna
added,nemeisho
to,duo
printed,eituasha
materials,inkishui
Listeners,Ilmeeta e radio
win,neitodol
airtime,airtime
for,duo
answering,answering
civic,enkopitoki
questions,questions
Tell,Enkarna nemeisho
a,a
true,enkarna
story,enkarna
of,le
a,a
county,enkata
election,esileto
Stories,Intitoi
show,nemeisho
collective,ileng’oni
gender,ishiligi nkiyena
action,entoki
Community,Entitoi le enkop
guide,guide
translated,neitobir
in,duo
3,eishumi
languages,eishumi
Game,Enkidong’oi
teaches,nemeisho
civic,enkopitoki
concepts,entoki
through,through
play,play
Focus,Entoki sidai
on,duo
justice,sheria
access,entoki
and,duo
human,iltunganak
rights,entoki leengai
Shows,Nemeisho
young,narua
musicians,ilopito
promoting,promoting
civic,enkopitoki
action,entoki
Used,Nemeisho
in,duo
Mandera,Mandera
civic,enkopitoki
sensitization,entoki
events,empukunyik
Interactive,Inkishui e sidai
panels,panels
and,duo
legal,sheria
kits,inkishui
Participants,Iltunganak
earn,neitodol
civic,enkopitoki
merit,intoki e sidai
points,intoki e sidai
Short,naibor
clips,intitoi
explain,naibor
rights,entoki leengai
simply,simply
Clubs,Ilopi
manage,neitoliki
elections,enkarna
and,duo
debates,ilopito e sidai
Winning,Intitoi neitodol
entries,entries
published,neitobir
in,duo
Civic,enkopitoki
Digest,enkopitoki
Collaboration,Enkiyieu
with,duo
urban,olchekut
artists,ilopito
Game,Enkidong’oi
piloted,neitodol
in,duo
Kisii,Kisii
and,duo
Eldoret,Eldoret
Matatus,Matatu
show,nemeisho
civic,enkopitoki
messages,enkarna
on,duo
windows,e-windows
Includes,Nemeisho
short,naibor
illustrated,neitobir
stories,intitoi
Workshops,Intoki e sidai
held,neitodol
in,duo
Nairobi,Nairobi
and,duo
Kisumu,Kisumu
Videos,Video
show,nemeisho
parts,entoki
of,le
the,the
Constitution,Katiba
Game,Enkidong’oi
distributed,neitobir
to,duo
schools,esoma
and,duo
youth,narua
groups,iltunganak
Youth,narua
show,nemeisho
how,enkurma
they,they
perform,enkurma
civic,enkopitoki
duties,duties
Students,nkera
reflect,nemeisho
on,on
self-governance,self-governance
Published,Nemeitobir
in,duo
daily,daily
newspapers,newspapers
App,enkore
has,nemeisho
logic,logic
games,inkidong’oi
on,on
civic,enkopitoki
themes,themes
Residents,Ilmang’atak
raise,neisho
questions,ramat
on,on
constitutional,Katiba
rights,entoki leengai
Focus,Entoki sidai
on,duo
emerging,neitobir
civic,enkopitoki
issues,issues
Targeting,entoki e sidai
primary,sidai
school,nkera
students,iltunganak
Focus,Entoki sidai
on,duo
language,enkiyieu
and,duo
gender,ishiligi nkiyena
equity,entoki le ishiligi nkiyena
Shows,Nemeisho
real,real
civic,enkopitoki
action,enkopitoki
in,duo
Turkana,Turkana
Magazine,Enkiteng’oto
includes,nemeisho
games,inkidong’oi
SMS,SMS
and,duo
call,call
support,support
in,duo
3,eishumi
languages,eishumi
Focus,Entoki sidai
on,duo
land,ng’ejuk
rights,entoki leengai
and,duo
justice,sheria
trials,trials
Designated,Designated
calls,calls
conducted,neitodol
in,duo
Nyeri,Nyeri
Students,Ilkera
engage,nemeisho
civic,enkopitoki
mentors,mentors
Distributed,Nemeitobir
via,duo
disability,olopolosho
councils,enkata
"""

# The data for the main dataset is also embedded in the code.
sentences_data = """#,Kiswahili,English Translation,Maa (Maasai) Translation (approximate)
1,IEBC inatoa kampeni ya kuongeza usajili wa wapiga kura katika kaunti zote.,IEBC is conducting a campaign to increase voter registration in all counties.,IEBC nemeishori entoki sidai duo entoki enkata esileto sidai olchani lemeita.
2,EACC inahimiza raia juu ya kuripoti ufisadi kupitia vikao vya kaunti.,EACC encourages citizens to report corruption through county forums.,EACC nemeishori enkurma enkopitoki iltunganak intae iloito esileto olchani.
3,KNCHR inafanya uhamishaji kuelimisha umma juu ya haki za kikatiba.,KNCHR carries out awareness to educate the public about constitutional rights.,KNCHR nemepesho entoki sidai duo iltunganak sidai enkatitin leengai.
4,NCIC inaleta mipango ya amani kuzuia vurugu za uchaguzi.,NCIC brings peace initiatives to prevent election violence.,NCIC nemeishori empiron enyor sidai duo enkopitoki ilkitok lolarashu.
5,Baraza la Magavana linasimamia vikao vya uhamasishaji juu ya huduma.,Council of Governors oversees forums about planned services.,Olmarei le Governors nemeitoliki ilinkopitoki duo entoki sidai naidimuno enkop.
6,Hazina ya Kitaifa inazindua mafunzo ya uandishi wa bajeti ili kukuza uwazi.,National Treasury launches training on budget writing to promote transparency.,Enkata oleng'amua nemeitat entoki esoma duosidai enkatitin ena budget duo esidai nabo.
7,Ajenda ya vijana inazindua mpango wa kuongeza ushiriki wa vijana katika utawala.,Youth Agenda launches a plan to increase youth participation in governance.,Ajenda le narua nemeitat empiron duo duo enarua duo enkerai iltunganak.
8,NGEC inakuza mazungumzo ya umma juu ya usawa wa kijinsia.,NGEC promotes public dialogues on gender equality.,NGEC nemeishori ilopito iltunganak duo iltunganak ilmeeta enkopitoki enkabare leishon.
9,Taasisi ya Katiba inaendesha kampeni ya elimu ya raia juu ya haki.,The Constitution Institute runs a civic education campaign on rights.,Taasisi le Katiba nemepesho entoki sidai duo esoma entoki enkata leengai.
10,IEBC inakuza majukwaa ya dijiti kwa kuelimisha wapiga kura wapya.,IEBC promotes digital platforms to educate new voters.,IEBC nemeishori enkiyio e-digital duo esoma iltunganak le esileto sidai.
11,Kampeni za EACC za kuhamasisha ripoti isiyojulikana ya kesi za ufisadi.,EACC campaigns to promote anonymous reporting of corruption cases.,EACC nemeishori entoki duo enkining’ori enkopitoki inkidung’u enkarna.
12,KNCHR inaelimisha raia juu ya ujumuishaji wa jamii zilizotengwa.,KNCHR educates citizens on inclusion of marginalized communities.,KNCHR nemepesho iltunganak duo esoma ilchan oleng’en lele enkop.
13,NCIC inasimamia vikao vya kujenga amani kupitia hafla za kitamaduni za jamii.,NCIC oversees peacebuilding forums via cultural community events.,NCIC nemeitoliki ilinkopitoki duo enyor sidai duo enkiyieu enkishu olmarei.
14,Baraza la Magavana linatoa elimu ya raia juu ya jinsi bajeti za kaunti zinavyofanya kazi.,Governors give civic education on how county budgets work.,Olmarei le Governors nemepesho entoki sidai duo esoma enkatitin le enkata le county.
15,Hazina ya Kitaifa hupanga mipango ya uhamasishaji juu ya ushuru na kufuata.,Treasury plans sensitization programs on taxation and compliance.,Enkata oleng’amua nemeisho empiron duo enkatitin ena ushuru duo enkerai.
16,Ajenda ya vijana inawapa viongozi vijana na ustadi wa elimu ya raia.,Youth Agenda equips young leaders with civic education skills.,Ajenda le narua nemeishori iltunganak narua duo enkatitin le entoki sidai.
17,Mawakili wa NGEC wa uwakilishi sawa wa wanawake katika machapisho ya uchaguzi.,NGEC advocates for equal representation of women in elections.,NGEC nemeitoliki enkurma duo entoki sidai duo enkatitin le ishiligi nkiyena.
18,Taasisi ya Katiba inaelimisha raia juu ya jukumu lao katika ushiriki wa umma.,The Constitution Institute educates on public participation roles.,Taasisi le Katiba nemepesho iltunganak duo esoma enkurma enkopitoki enkanashe.
19,IEBC hupanga vikao juu ya uadilifu wa uchaguzi na utayari wa 2025.,IEBC organizes forums on electoral integrity and 2025 readiness.,IEBC nemeitoliki ilinkopitoki duo enkatitin le enkarna duo entoki duo duo 2025.
20,EACC huunda vikundi vya walinzi wa ndani kupigana na hongo katika kaunti.,EACC forms local watchdog groups to fight bribery.,EACC nemeitodol ileng’oni le sidai duo emunyunye enkarna esileto county.
21,KNCHR huanza vilabu kufundisha wanafunzi haki zao za kikatiba.,KNCHR starts clubs to teach students their constitutional rights.,KNCHR nemeitat ilopi le nkera duo esoma iltunganak le nkera duo entoki leengai.
22,NCIC hutumia misafara ya amani kueneza ujumbe wa umoja kabla ya uchaguzi.,NCIC uses peace caravans to spread unity before elections.,NCIC nemeisho enkarna e sidai duo duo empukunyik duo enkata enkarna.
23,Baraza la Magavana linazindua Programu ya Uhamasishaji wa Marekebisho.,Council of Governors launches Reform Awareness Program.,Olmarei le Governors nemeitat empiron e sidai duo duo enkiyieu enkishui.
24,Hazina ya Kitaifa hutumia zana zinazoingiliana kuelezea bajeti.,Treasury uses interactive tools to explain budgets.,Enkata oleng’amua nemeisho inkishui enkatitin duo duo entoki naibor.
25,Ajenda ya vijana inaleta Hackathons kubuni sera za vijana.,Youth Agenda introduces hackathons to design youth policies.,Ajenda le narua nemeisho ilopito enkatitin duo duo duo duo inkishui enarua.
26,NGEC inasaidia wanawake kupitia ushauri katika michakato ya kisiasa.,NGEC supports women via political mentorship programs.,NGEC nemeisho ishiligi nkiyena duo entoki e sidai duo duo empiron le politika.
27,Taasisi ya Katiba inazalisha podcasts kueneza ufahamu wa katiba.,The Constitution Institute produces podcasts to spread awareness.,Taasisi le Katiba nemeisho ilopito duo duo duo duo duo duo entoki leengai.
28,Washirika wa IEBC na watangazaji kwa maudhui ya uhamasishaji wa wapiga kura.,IEBC partners with media for voter sensitization content.,IEBC nemeisho enkiyieu olchani duo duo ilasigirori duo duo empiron le esileto.
29,EACC inaleta njia nyingi za kuripoti kwa wazungu.,EACC brings multiple channels for anonymous reporting.,EACC nemeisho ilchan ileng’oni duo duo entoki e sidai duo enkining’ori.
30,KNCHR inakuza elimu ya raia kwa watu wenye ulemavu.,KNCHR promotes civic education for people with disabilities.,KNCHR nemeishori entoki sidai duo iltunganak ilmeeta entoki olopolosho.
31,NCIC huunda kamati mpya za amani kushughulikia mizozo ya mitaa.,NCIC forms new peace committees to address local conflicts.,NCIC nemeitat ilchani le sidai duo duo duo enkerai enkopitoki.
32,Baraza la Magavana linajaribu vituo vya mitaa kwa ujifunzaji wa kujitolea.,Council of Governors tests local hubs for volunteer learning.,Olmarei le Governors nemeisho enkoito duo duo ilchani le nkera e sidai.
33,Hazina ya Kitaifa inajibu maswali ya raia juu ya mgao wa bajeti.,Treasury responds to citizen questions on budget allocations.,Enkata oleng’amua nemeisho iramat iltunganak duo entoki enkata le budget.
34,Ajenda ya vijana inaleta tiktok kufundisha uwajibikaji wa raia.,Youth Agenda uses TikTok to teach civic responsibility.,Ajenda le narua nemeisho TikTok duo esoma entoki sidai leengai.
35,Mawakili wa NGEC wa mgao sawa katika bajeti za kitaifa na kata.,NGEC advocates for gender-equal budgeting at national and ward levels.,NGEC nemeisho empiron le ishiligi nkiyena duo duo duo enkatitin le enkata duo enche.
36,Taasisi ya Katiba hutoa vijitabu juu ya haki na majukumu.,Constitution Institute provides booklets on rights and duties.,Taasisi le Katiba nemeisho intitoi duo duo entoki leengai duo enkurma.
37,IEBC hutumia mitandao ya boda boda kueneza habari ya usajili wa wapigakura.,IEBC uses boda boda networks to spread voter registration info.,IEBC nemeisho enkiyieu e-boda boda duo duo duo enkiyieu entoki esileto.
38,EACC inafanya kazi na asasi za kiraia kutekeleza kanuni za kupambana na ufisadi.,EACC partners with civil groups to enforce anti-corruption rules.,EACC nemeisho inkera e sidai duo duo duo duo duo enkarna esileto enkopitoki.
39,KNCHR inazindua dawati la msaada wa kisheria kwa jamii zilizotengwa.,KNCHR launches legal help desks for marginalized communities.,KNCHR nemeitat empasurur enkai duo ilchan lele enkop.
40,NCIC inajumuisha masomo ya amani katika programu za shule.,NCIC includes peace studies in school programs.,NCIC nemeisho entoki sidai le amani duo duo duo entoki e sidai le nkera.
41,Baraza la Magavana linaendesha picha za barabarani juu ya ufahamu wa uamuzi.,Council of Governors runs billboards to raise awareness.,Olmarei le Governors nemeisho ilasigirori e sidai duo enkatitin le esomoni.
42,Hazina huunda programu ya kufundisha raia juu ya fedha za umma.,Treasury creates programs to educate citizens on public funds.,Enkata oleng’amua nemeitat empiron e sidai duo duo duo enkata oleng.
43,Ajenda ya vijana hufundisha viongozi wa vijana juu ya kanuni za utawala.,Youth Agenda teaches youth leaders on governance principles.,Ajenda le narua nemepesho iltunganak narua duo entoki e sidai duo duo duo enkata.
44,NGEC hupanga matukio ya kiwango cha kaunti juu ya umoja wa kijinsia.,NGEC organizes county-level events on gender unity.,NGEC nemeitoliki enkiyieu le county duo duo duo entoki le ishiligi nkiyena.
45,Taasisi ya Katiba inaleta kujifunza e kwa maarifa ya raia.,Constitution Institute introduces e-learning for civic knowledge.,Taasisi le Katiba nemeitat enkiyieu e-learning duo entoki leengai.
46,IEBC inaelimisha umma juu ya mageuzi mapya katika mifumo ya uchaguzi.,IEBC educates the public on new electoral reforms.,IEBC nemepesho iltunganak duo entoki e sidai le enkarna duo duo duo enkarna sidai.
47,EACC huunda vilabu vya shule ili kukuza maadili ya kupambana na ufisadi.,EACC forms school clubs to promote anti-corruption ethics.,EACC nemeitat ilopi le nkera duo duo entoki sidai duo duo duo enkarna.
48,KNCHR inasimamia mazungumzo ya kaunti juu ya haki na usawa.,KNCHR oversees county dialogues on rights and equality.,KNCHR nemeitoliki enkopitoki le county duo duo entoki leengai duo entoki sidai.
49,NCIC hutumia vyombo vya habari kukatisha tamaa hotuba ya chuki katika uchaguzi.,NCIC uses media to discourage hate speech during elections.,NCIC nemeisho olchani duo duo duo entoki sidai duo duo duo enkarna.
50,Baraza la Magavana huunda mfululizo wa sauti juu ya maswala ya kujitolea.,Governors create podcast series on volunteer issues.,Olmarei le Governors nemeisho ilasigirori e sidai duo empiron le inkera e sidai.
51,Kampeni za EACC za kuhamasisha ripoti isiyojulikana ya kesi za ufisadi.,EACC campaigns to promote anonymous reporting of corruption cases.,EACC nemeishori entoki duo enkining’ori enkopitoki inkidung’u enkarna.
52,KNCHR inaelimisha raia juu ya ujumuishaji wa jamii zilizotengwa.,KNCHR educates citizens on inclusion of marginalized communities.,KNCHR nemepesho iltunganak duo esoma ilchan oleng’en lele enkop.
53,NCIC inasimamia vikao vya kujenga amani kupitia hafla za kitamaduni za jamii.,NCIC oversees peacebuilding forums via cultural community events.,NCIC nemeitoliki ilinkopitoki duo enyor sidai duo enkiyieu enkishu olmarei.
54,Baraza la Magavana linatoa elimu ya raia juu ya jinsi bajeti za kaunti zinavyofanya kazi.,Governors give civic education on how county budgets work.,Olmarei le Governors nemepesho entoki sidai duo esoma enkatitin le enkata le county.
55,Hazina ya Kitaifa hupanga mipango ya uhamasishaji juu ya ushuru na kufuata.,Treasury plans sensitization programs on taxation and compliance.,Enkata oleng’amua nemeisho empiron duo enkatitin ena ushuru duo enkerai.
56,Ajenda ya vijana inawapa viongozi vijana na ustadi wa elimu ya raia.,Youth Agenda equips young leaders with civic education skills.,Ajenda le narua nemeishori iltunganak narua duo enkatitin le entoki sidai.
57,Mawakili wa NGEC wa uwakilishi sawa wa wanawake katika machapisho ya uchaguzi.,NGEC advocates for equal representation of women in elections.,NGEC nemeitoliki enkurma duo entoki sidai duo enkatitin le ishiligi nkiyena.
58,Taasisi ya Katiba inaelimisha raia juu ya jukumu lao katika ushiriki wa umma.,The Constitution Institute educates on public participation roles.,Taasisi le Katiba nemepesho iltunganak duo esoma enkurma enkopitoki enkanashe.
59,IEBC hupanga vikao juu ya uadilifu wa uchaguzi na utayari wa 2025.,IEBC organizes forums on electoral integrity and 2025 readiness.,IEBC nemeitoliki ilinkopitoki duo enkatitin le enkarna duo entoki duo duo 2025.
60,EACC huunda vikundi vya walinzi wa ndani kupigana na hongo katika kaunti.,EACC forms local watchdog groups to fight bribery.,EACC nemeitodol ileng’oni le sidai duo emunyunye enkarna esileto county.
61,KNCHR huanza vilabu kufundisha wanafunzi haki zao za kikatiba.,KNCHR starts clubs to teach students their constitutional rights.,KNCHR nemeitat ilopi le nkera duo esoma iltunganak le nkera duo entoki leengai.
62,NCIC hutumia misafara ya amani kueneza ujumbe wa umoja kabla ya uchaguzi.,NCIC uses peace caravans to spread unity before elections.,NCIC nemeisho enkarna e sidai duo duo empukunyik duo enkata enkarna.
63,Baraza la Magavana linazindua Programu ya Uhamasishaji wa Marekebisho.,Council of Governors launches Reform Awareness Program.,Olmarei le Governors nemeitat empiron e sidai duo duo enkiyieu enkishui.
64,Hazina ya Kitaifa hutumia zana zinazoingiliana kuelezea bajeti.,Treasury uses interactive tools to explain budgets.,Enkata oleng’amua nemeisho inkishui enkatitin duo duo entoki naibor.
65,Ajenda ya vijana inaleta Hackathons kubuni sera za vijana.,Youth Agenda introduces hackathons to design youth policies.,Ajenda le narua nemeisho ilopito enkatitin duo duo duo duo inkishui enarua.
66,NGEC inasaidia wanawake kupitia ushauri katika michakato ya kisiasa.,NGEC supports women via political mentorship programs.,NGEC nemeisho ishiligi nkiyena duo entoki e sidai duo duo empiron le politika.
67,Taasisi ya Katiba inazalisha podcasts kueneza ufahamu wa katiba.,The Constitution Institute produces podcasts to spread awareness.,Taasisi le Katiba nemeisho ilopito duo duo duo duo duo duo entoki leengai.
68,Washirika wa IEBC na watangazaji kwa maudhui ya uhamasishaji wa wapiga kura.,IEBC partners with media for voter sensitization content.,IEBC nemeisho enkiyieu olchani duo duo ilasigirori duo duo empiron le esileto.
69,EACC inaleta njia nyingi za kuripoti kwa wazungu.,EACC brings multiple channels for anonymous reporting.,EACC nemeisho ilchan ileng’oni duo duo entoki e sidai duo enkining’ori.
70,KNCHR inakuza elimu ya raia kwa watu wenye ulemavu.,KNCHR promotes civic education for people with disabilities.,KNCHR nemeishori entoki sidai duo iltunganak ilmeeta entoki olopolosho.
71,NCIC huunda kamati mpya za amani kushughulikia mizozo ya mitaa.,NCIC forms new peace committees to address local conflicts.,NCIC nemeitat ilchani le sidai duo duo duo enkerai enkopitoki.
72,Baraza la Magavana linajaribu vituo vya mitaa kwa ujifunzaji wa kujitolea.,Council of Governors tests local hubs for volunteer learning.,Olmarei le Governors nemeisho enkoito duo duo ilchani le nkera e sidai.
73,Hazina ya Kitaifa inajibu maswali ya raia juu ya mgao wa bajeti.,Treasury responds to citizen questions on budget allocations.,Enkata oleng’amua nemeisho iramat iltunganak duo entoki enkata le budget.
74,Ajenda ya vijana inaleta tiktok kufundisha uwajibikaji wa raia.,Youth Agenda uses TikTok to teach civic responsibility.,Ajenda le narua nemeisho TikTok duo esoma entoki sidai leengai.
75,Mawakili wa NGEC wa mgao sawa katika bajeti za kitaifa na kata.,NGEC advocates for gender-equal budgeting at national and ward levels.,NGEC nemeisho empiron le ishiligi nkiyena duo duo duo enkatitin le enkata duo enche.
76,Taasisi ya Katiba hutoa vijitabu juu ya haki na majukumu.,Constitution Institute provides booklets on rights and duties.,Taasisi le Katiba nemeisho intitoi duo duo entoki leengai duo enkurma.
77,IEBC hutumia mitandao ya boda boda kueneza habari ya usajili wa wapigakura.,IEBC uses boda boda networks to spread voter registration info.,IEBC nemeisho enkiyieu e-boda boda duo duo duo enkiyieu entoki esileto.
78,EACC inafanya kazi na asasi za kiraia kutekeleza kanuni za kupambana na ufisadi.,EACC partners with civil groups to enforce anti-corruption rules.,EACC nemeisho inkera e sidai duo duo duo duo duo enkarna esileto enkopitoki.
79,KNCHR inazindua dawati la msaada wa kisheria kwa jamii zilizotengwa.,KNCHR launches legal help desks for marginalized communities.,KNCHR nemeitat empasurur enkai duo ilchan lele enkop.
80,NCIC inajumuisha masomo ya amani katika programu za shule.,NCIC includes peace studies in school programs.,NCIC nemeisho entoki sidai le amani duo duo duo entoki e sidai le nkera.
81,Baraza la Magavana linaendesha picha za barabarani juu ya ufahamu wa uamuzi.,Council of Governors runs billboards to raise awareness.,Olmarei le Governors nemeisho ilasigirori e sidai duo enkatitin le esomoni.
82,Hazina huunda programu ya kufundisha raia juu ya fedha za umma.,Treasury creates programs to educate citizens on public funds.,Enkata oleng’amua nemeitat empiron e sidai duo duo duo enkata oleng.
83,Ajenda ya vijana hufundisha viongozi wa vijana juu ya kanuni za utawala.,Youth Agenda teaches youth leaders on governance principles.,Ajenda le narua nemepesho iltunganak narua duo entoki e sidai duo duo duo enkata.
84,NGEC hupanga matukio ya kiwango cha kaunti juu ya umoja wa kijinsia.,NGEC organizes county-level events on gender unity.,NGEC nemeitoliki enkiyieu le county duo duo duo entoki le ishiligi nkiyena.
85,Taasisi ya Katiba inaleta kujifunza e kwa maarifa ya raia.,Constitution Institute introduces e-learning for civic knowledge.,Taasisi le Katiba nemeitat enkiyieu e-learning duo entoki leengai.
86,IEBC inaelimisha umma juu ya mageuzi mapya katika mifumo ya uchaguzi.,IEBC educates the public on new electoral reforms.,IEBC nemepesho iltunganak duo entoki e sidai le enkarna duo duo duo enkarna sidai.
87,EACC huunda vilabu vya shule ili kukuza maadili ya kupambana na ufisadi.,EACC forms school clubs to promote anti-corruption ethics.,EACC nemeitat ilopi le nkera duo duo entoki sidai duo duo duo enkarna.
88,KNCHR inasimamia mazungumzo ya kaunti juu ya haki na usawa.,KNCHR oversees county dialogues on rights and equality.,KNCHR nemeitoliki enkopitoki le county duo duo entoki leengai duo entoki sidai.
89,NCIC hutumia vyombo vya habari kukatisha tamaa hotuba ya chuki katika uchaguzi.,NCIC uses media to discourage hate speech during elections.,NCIC nemeisho olchani duo duo duo entoki sidai duo duo duo enkarna.
90,Baraza la Magavana huunda mfululizo wa sauti juu ya maswala ya kujitolea.,Governors create podcast series on volunteer issues.,Olmarei le Governors nemeisho ilasigirori e sidai duo empiron le inkera e sidai.
91,Hazina ya Kitaifa hupanga wavuti juu ya uwajibikaji wa fedha za umma.,Treasury organizes webinars on public finance accountability.,Enkata oleng’amua nemeitoliki enkiyieu e-webinar duo enkatitin le enkata oleng.
92,Ajenda ya vijana hufundisha vijana kutumia teknolojia kwa ushiriki wa raia.,Youth Agenda trains youth to use tech for civic engagement.,Ajenda le narua nemepesho iltunganak narua duo enkiyieu e-technology duo entoki e sidai leengai.
93,NGEC inachukua watunga sheria juu ya sheria nyeti za kijinsia.,NGEC trains lawmakers on gender-sensitive laws.,NGEC nemepesho iltunganak le sheria duo entoki sidai le ishiligi nkiyena.
94,Taasisi ya Katiba inaunda michoro kwa kufikia elimu ya raia.,Constitution Institute creates illustrations to aid civic education.,Taasisi le Katiba nemeisho ilasigirori duo duo duo duo entoki sidai leengai.
95,IEBC inazindua mipango ya usajili wa wapiga kura wa Diaspora.,IEBC launches voter registration plans for the diaspora.,IEBC nemeitat empiron e sidai duo duo entoki esileto le esiligi e nenkop.
96,EACC inaleta programu ya rununu kwa ripoti rahisi ya ufisadi.,EACC introduces a mobile app for easy corruption reporting.,EACC nemeisho enkore e-mobile duo entoki enkining’ori e sidai le enkarna.
97,KNCHR inafundisha washirika kusaidia jamii juu ya haki za kisheria.,KNCHR trains partners to help communities on legal rights.,KNCHR nemepesho inkera e sidai duo ilchan lele enkop duo entoki leengai.
98,Washirika wa NCIC na vikundi vya kidini kukuza amani.,NCIC partners with religious groups to promote peace.,NCIC nemeisho enkiyieu inkera e sidai duo iltunganak le sidai.
99,Baraza la Magavana linafungua vibanda vya elimu ya raia katika maeneo ya vijijini.,Governors open civic education booths in rural areas.,Olmarei le Governors nemeisho intulie enkopitoki e sidai le ilchan le neituashu.
100,Hazina inaleta zana za mkondoni kwa ufuatiliaji wa fedha za umma.,Treasury provides online tools to track public funds.,Enkata oleng’amua nemeisho enkiyieu e-online duo entoki e sidai le enkata oleng.
101,Ajenda ya vijana inashauri vijana juu ya utawala na jukumu la raia.,Youth Agenda advises youth on governance and civic duty.,Ajenda le narua nemeisho iltunganak narua duo entoki e sidai leengai duo enkata.
102,NGEC inaelimisha umma juu ya ripoti ya GBV na tiba za kisheria.,NGEC educates public on GBV reporting and legal redress.,NGEC nemepesho iltunganak duo entoki e sidai le GBV duo entoki e sidai le sheria.
103,Taasisi ya Katiba hutoa hati juu ya haki za raia.,Constitution Institute provides documents on citizen rights.,Taasisi le Katiba nemeisho intitoi duo entoki leengai le iltunganak.
104,IEBC inashirikiana na vyuo vikuu juu ya utafiti wa elimu ya wapiga kura.,IEBC collaborates with universities on voter education research.,IEBC nemeisho enkiyieu e-university duo entoki e sidai le esoma le esileto.
105,EACC inawapa watendaji wa vyombo vya habari na ustadi wa kuripoti ufisadi.,EACC trains media professionals on corruption reporting.,EACC nemepesho ilasigirori duo entoki e sidai le enkarna.
106,KNCHR hutoa misaada ya kisheria ya bure wakati wa vikao vya umma.,KNCHR offers free legal aid during public forums.,KNCHR nemeisho entoki le sheria duo duo e sidai enkopitoki.
107,NCIC inafundisha wapatanishi wa ndani kwa utatuzi wa migogoro.,NCIC trains local mediators in conflict resolution.,NCIC nemepesho ileng’oni le sidai duo enkatitin le emanyata.
108,Baraza la Magavana linatoa yaliyomo kwenye E-Kujifunza juu ya Devolution.,Governors provide e-learning content on devolution.,Olmarei le Governors nemeisho enkiyieu e-learning duo entoki e sidai le devolushon.
109,Hazina inafundisha raia juu ya kutumia data ya bajeti wazi.,Treasury trains citizens on using open budget data.,Enkata oleng’amua nemepesho iltunganak duo enkiyieu entoki le budget.
110,Ajenda ya vijana inapeana changamoto za vijana kuunda suluhisho za teknolojia ya raia.,Youth Agenda challenges youth to create civic tech solutions.,Ajenda le narua nemeisho enkarna e sidai duo iltunganak narua duo enkiyieu e-technology leengai.
111,Kampeni za NGEC za uwakilishi wa wanawake katika makusanyiko ya kaunti.,NGEC campaigns for women’s representation in county assemblies.,NGEC nemeishori entoki sidai le ishiligi nkiyena duo enkata le county.
112,Taasisi ya Katiba inaboresha programu yake kwa elimu ya raia.,Constitution Institute improves its civic education programs.,Taasisi le Katiba nemeishori empiron le entoki sidai.
113,IEBC hutumia watendaji wa media ya kijamii kufikia wapiga kura wachanga.,IEBC uses social media influencers to reach young voters.,IEBC nemeisho ilasigirori e-social media duo esoma iltunganak narua.
114,EACC inasaidia ofisi za mitaa kuzuia utumiaji mbaya wa fedha za umma.,EACC supports local offices in preventing misuse of public funds.,EACC nemeisho enkopitoki e neituashu duo entoki sidai le enkata oleng.
115,KNCHR hutumia ukumbi wa michezo kufundisha kanuni za haki za binadamu.,KNCHR uses theatre to teach human rights principles.,KNCHR nemeisho ilopito e-oltoyi duo entoki leengai le iltunganak.
116,NCIC hutuma mabalozi wa amani katika kaunti zinazokabiliwa na migogoro.,NCIC sends peace ambassadors to conflict-prone counties.,NCIC nemeisho ileng’oni le sidai duo duo enkata lemeishori esikirie.
117,Baraza la Magavana hutumia podcasts kwa ufahamu wa umma.,Governors use podcasts to raise public awareness.,Olmarei le Governors nemeisho enkiyieu e-podcast duo entoki sidai leengai.
118,Hazina inaongoza kujifunza bajeti kwa watazamaji vijana.,Treasury leads budget learning for youth audiences.,Enkata oleng’amua nemepesho entoki le budget duo iltunganak narua.
119,Ajenda ya vijana inawapa kaunti na halmashauri za vijana zinazotumika.,Youth Agenda empowers counties and active youth councils.,Ajenda le narua nemeisho inkera duo iltunganak le county duo halmashauri le narua.
120,NGEC hupanga mikutano ya kitaifa ya utawala wa pamoja.,NGEC organizes national meetings on inclusive governance.,NGEC nemeitoliki ilinkopitoki le Kenya duo entoki enkata e sidai enkopitoki.
121,Taasisi ya Katiba huunda zana za jaribio ili kufanya kujifunza kuwa maingiliano.,Constitution Institute creates interactive tools for learning.,Taasisi le Katiba nemeisho inkishui e sidai duo duo duo entoki e-learning.
122,IEBC inaajiri vijana wa kujitolea kwa kufikia raia.,IEBC recruits youth volunteers for civic outreach.,IEBC nemeisho iltunganak narua duo duo duo esoma iltunganak.
123,EACC inaleta tuzo za kuripoti ufisadi.,EACC introduces awards for reporting corruption.,EACC nemeisho enkiyieu e-tuzo duo enkining’ori e sidai le enkarna.
124,KNCHR inatangaza ushauri wa kisheria na mada za raia kwenye redio.,KNCHR airs legal advice and civic topics on radio.,KNCHR nemeisho enkiyieu e-radio duo entoki leengai le iltunganak.
125,NCIC hutumia sanaa kukuza ujumbe wa kitaifa wa mshikamano.,NCIC uses art to promote national unity messaging.,NCIC nemeisho ilopito e-sanaa duo duo entoki sidai le ilchani Kenya.
126,Baraza la Magavana Washirika na Waandishi wa Habari kwa Habari za Devolution.,Governors partner with journalists for devolution news.,Olmarei le Governors nemeisho enkiyieu ilasigirori duo duo entoki e devolushon.
127,Hazina inazindua Chatbot iliyo na nguvu ya AI kwa maswali ya bajeti.,Treasury launches AI-powered chatbot for budget questions.,Enkata oleng’amua nemeitat enkore e-chatbot e AI duo entoki e sidai le budget.
128,Ajenda ya vijana hupanga hafla za marathon kwa ufahamu wa wapiga kura.,Youth Agenda organizes marathons for voter awareness.,Ajenda le narua nemeitoliki empukunyik le marathon duo entoki sidai le esileto.
129,NGEC inaunda kampeni za media za kijamii kwa usawa wa kijinsia.,NGEC creates social media campaigns on gender equality.,NGEC nemeisho enkiyieu e-social media duo entoki le ishiligi nkiyena.
130,Taasisi ya Katiba inachanganya habari potofu na yaliyomo ukweli.,Constitution Institute counters disinformation with facts.,Taasisi le Katiba nemeisho entoki sidai duo duo duo duo duo duo enkarna.
131,IEBC inaunda michezo inayoingiliana kufundisha michakato ya uchaguzi.,IEBC develops interactive games to teach election processes.,IEBC nemeisho inkidong’oi e sidai duo entoki le enkarna.
132,EACC inachukua ujumbe wa kupambana na rushwa kwa kaunti zote.,EACC takes anti-corruption message to all counties.,EACC nemeisho entoki sidai le enkarna duo enkata oleng.
133,KNCHR inazindua vituo vya mkondoni kwa elimu ya haki za binadamu.,KNCHR launches online platforms for human rights education.,KNCHR nemeitat enkiyieu e-online duo entoki leengai le iltunganak.
134,NCIC huunda nafasi za mkondoni kwa mazungumzo ya utatuzi wa migogoro.,NCIC creates online spaces for conflict resolution dialogues.,NCIC nemeitat enkiyieu e-online duo enkopitoki le emanyata.
135,Baraza la Magavana hulipa kaunti bora katika elimu ya raia.,Governors reward best counties in civic education.,Olmarei le Governors nemeisho enkiyieu e-tuzo duo iltunganak le enkata sidai.
136,Hazina inafundisha waandishi wa habari juu ya kuchambua data ya bajeti.,Treasury trains journalists on analyzing budget data.,Enkata oleng’amua nemepesho ilasigirori duo entoki le budget.
137,Ajenda ya vijana huiga vikao vya kisheria vya kujifunza vijana.,Youth Agenda simulates legal sessions for youth learning.,Ajenda le narua nemeisho ilopito le sheria duo esoma iltunganak narua.
138,NGEC hupima maendeleo ya usawa wa kijinsia katika vitengo vilivyowekwa.,NGEC monitors gender equity progress in targeted sectors.,NGEC nemeitoliki entoki sidai le ishiligi nkiyena duo enkata.
139,Taasisi ya Katiba inafadhili matukio ya uvumbuzi wa teknolojia.,Constitution Institute funds civic tech innovation events.,Taasisi le Katiba nemeisho enkiyieu e-sidaitin e-technology.
140,IEBC hutumia kampeni za mlango hadi mlango kwa ufahamu wa wapiga kura wa mbali.,IEBC uses door-to-door campaigns to reach remote voters.,IEBC nemeishori empiron e sidai duo duo duo iltunganak le neituashu.
141,Kujadili uhuru wa raia katika nafasi ya dijiti.,Discussing citizen freedom in digital space.,Entoki e sidai le iltunganak duo enkiyieu e-digital.
142,Programu inatoa vidokezo vya kujifunza na kuripoti.,The app gives learning and reporting tips.,Enkore nemeisho entoki e sidai le esoma duo enkining’ori.
143,Elimu ya wapiga kura kwa vikundi vilivyotengwa.,Voter education for marginalized groups.,Entoki sidai le esileto duo ilchan lele enkop.
144,Hadithi zilizochorwa za ushiriki wa raia na wanawake.,Illustrated stories on civic participation and women.,Intitoi e sidai le iltunganak duo ishiligi nkiyena.
145,Ushiriki wa msingi wa Trivia juu ya sheria za Kenya.,Grassroots trivia participation on Kenyan laws.,Entoki e sidai le Trivia duo entoki le sheria le Kenya.
146,Nakala ya kisheria inayopatikana kwa wanafunzi na umma.,Legal text accessible to students and the public.,Intitoi le sheria neitobir enkera duo iltunganak.
147,Tabia inayotumika katika skirti za raia na vifaa.,Behavior reflected in civic skits and tools.,Enkisia neitobir duo entoki le skits duo inkishui.
148,Warsha zilizofanyika katika kaunti 6 za mvutano wa juu.,Workshops held in six high-tension counties.,Intoki e sidai nemetodol duo enkata enene e sidai duo e esikirie.
149,Inaonyesha historia ya uhuru wa raia nchini Kenya.,Depicts history of citizen freedom in Kenya.,Nemeisho entoki e sidai le iltunganak duo Kenya.
150,Ushirikiano wa vijana huangaza baada ya machapisho 3 ya virusi.,Youth collaboration highlighted after 3 viral posts.,Enkiyieu e narua nemeisho duo entoki sidai duo duo duo enkiyieu e viral.
151,Zingatia mbinu za utetezi wa nyasi.,Focus on grassroots advocacy strategies.,Entoki sidai duo ilchan le neituashu duo entoki e sidai.
152,Timu zinashindana juu ya maarifa ya uwajibikaji wa umma.,Teams compete on public accountability knowledge.,Ileng’oni neitodol duo entoki leengai le enkurma le enkata oleng.
153,Ujumbe uliolengwa uliotumwa kabla ya vikao vya kaunti.,Targeted message sent before county forums.,Enkarna neishori duo duo duo duo ilinkopitoki le county.
154,Vibanda vya raia vilivyowekwa katika Kakuma na Dadaab.,Civic booths set up in Kakuma and Dadaab.,Intulie enkopitoki nemeitodol duo Kakuma duo Dadaab.
155,PDF ni pamoja na sheria.,PDF includes legal guidelines.,PDF nemeisho intitoi le sheria.
156,Paint na vilabu vya raia na wasanii wa ndani.,Murals and civic clubs with local artists.,Inkidong’oi duo ilopi le enkopitoki duo ilopito le neituashu.
157,Tukio ni pamoja na mazungumzo ya raia na michezo.,Event includes civic talks and games.,Empukunyik nemeisho enkopitoki duo inkidong’oi.
158,Mafunzo ya media ya kijamii kwa waundaji wa bidhaa.,Social media training for content creators.,Empiron e sidai le media le social duo ilasigirori.
159,Mfululizo unafuata kikundi cha wapiga kura wa wanafunzi.,Series follows student voter group.,Enkarna nemeisho iltunganak le nkera le esileto.
160,Sehemu ya elimu ya raia juu ya uwajibikaji wa kifedha.,Civic education section on financial responsibility.,Entoki sidai le enkopitoki duo entoki enkata le esidai.
161,Takwimu zinaarifu mipango ya uhamasishaji wa kaunti.,Statistics inform county sensitization plans.,Intoki e sidai nemeisho empiron le county.
162,Mazungumzo ya wanafunzi juu ya ujana na nguvu za raia.,Student discussions on youth and civic power.,Inkopitoki le nkera duo entoki e narua duo entoki leengai le iltunganak.
163,Washiriki hutatua puzzles za raia.,Participants solve civic puzzles.,Iltunganak nemeisho enkidong’oi le enkopitoki.
164,Warsha za watetezi rasmi wa kisheria.,Workshops for certified legal advocates.,Intoki e sidai le iltunganak le sheria.
165,Nambari mpya iliyoongezwa kwa vifaa vya kuchapishwa.,New code added to printed materials.,Enkarna e sidai nemeisho inkishui le eituasha.
166,Wasikilizaji wanashinda wakati wa hewa kwa kujibu maswali ya raia.,Listeners win airtime for answering civic questions.,Ilmeeta e radio neitodol duo enkidong’oi le enkopitoki.
167,Simulia uchaguzi wa kweli wa kaunti ya maisha.,Tell a true story of a county election.,Enkarna nemeisho entoki le esileto enkata sidai.
168,Hadithi zinaonyesha hatua ya pamoja ya jinsia.,Stories show collective gender action.,Intitoi nemeisho entoki le ishiligi nkiyena duo duo ileng’oni.
169,Jumuia iliyotafsiriwa katika lugha 3.,Community guide translated in 3 languages.,Entitoi le enkop neitobir duo enkiyieu eishumi.
170,Mchezo hufundisha dhana za raia kupitia kucheza.,Game teaches civic concepts through play.,Enkidong’oi nemeisho entoki e sidai le enkopitoki.
171,Zingatia upatikanaji wa haki na haki za binadamu.,Focus on justice access and human rights.,Entoki sidai le entoki e sheria duo entoki leengai le iltunganak.
172,Inaonyesha wanamuziki wa vijana wanaokuza hatua za raia.,Shows young musicians promoting civic action.,Nemeisho ilopito le narua duo duo entoki e sidai le enkopitoki.
173,Inatumika katika hafla za uhamasishaji za Mandera Civic.,Used in Mandera civic sensitization events.,Nemeisho duo empukunyik le enkopitoki le Mandera.
174,Paneli zinazoingiliana na mikoba ya kisheria.,Interactive panels and legal kits.,Inkishui e sidai duo ilasigirori le sheria.
175,Washiriki hupata alama za sifa za raia.,Participants earn civic merit points.,Iltunganak neitodol duo intoki e sidai le enkopitoki.
176,Sehemu fupi zinaelezea haki kwa maneno rahisi.,Short clips explain rights simply.,Intitoi naibor nemeisho entoki leengai naibor.
177,Vilabu vinasimamia uchaguzi na mijadala.,Clubs manage elections and debates.,Ilopi neitoliki enkarna duo ilopito e sidai.
178,Viingilio vya kushinda vilivyochapishwa katika Civic Digest.,Winning entries published in Civic Digest.,Intitoi neitodol neitobir duo entitoi e sidai le enkopitoki.
179,Ushirikiano na wasanii wa mijini.,Collaboration with urban artists.,Enkiyieu duo ilopito le olchekut.
180,Mchezo uliyopigwa marubani huko Kisii na Eldoret.,Game piloted in Kisii and Eldoret.,Enkidong’oi neitodol enkishui Kisii duo Eldoret.
181,Matatus huonyesha ujumbe wa raia kwenye Windows.,Matatus show civic messages on windows.,Matatu nemeisho enkarna le enkopitoki duo e-windows.
182,Ni pamoja na hadithi fupi zilizochorwa.,Includes short illustrated stories.,Nemeisho intitoi naibor neitobir duo duo duo e sidai.
183,Warsha zilizofanyika jijini Nairobi na Kisumu.,Workshops held in Nairobi and Kisumu.,Intoki e sidai neitodol duo Nairobi duo Kisumu.
184,Video zinaonyesha sura za Katiba.,Videos show parts of the Constitution.,Video nemeisho entoki le Katiba.
185,Mchezo uliosambazwa kwa shule na vikundi vya vijana.,Game distributed to schools and youth groups.,Enkidong’oi neitobir duo esoma duo iltunganak narua.
186,Vijana wanaonyesha jinsi wanavyofanya majukumu ya raia.,Youth show how they perform civic duties.,Iltunganak narua nemeisho enkurma e sidai le enkopitoki.
187,Wanafunzi wanashiriki tafakari za utawala wa kibinafsi.,Students reflect on self-governance.,Iltunganak le nkera nemeisho entoki enkata le ninye.
188,Iliyotolewa katika magazeti ya kila siku.,Published in daily newspapers.,Nemeitobir duo enkiteng’oto le ilasigirori.
189,Programu ina michezo ya mantiki kulingana na mada za raia.,App has logic games on civic themes.,Enkore nemeisho inkidong’oi le sidai duo entoki e sidai le enkopitoki.
190,Wakazi huleta maswali juu ya haki za kikatiba.,Residents raise questions on constitutional rights.,Ilmang’atak neisho iramat duo entoki leengai le Katiba.
191,Zingatia maswala ya raia yanayowezekana.,Focus on emerging civic issues.,Entoki sidai le entoki le enkopitoki neitobir.
192,Kulengwa kwa wanafunzi wa shule ya msingi.,Targeting primary school students.,Entoki e sidai duo iltunganak le nkera le sidai.
193,Zingatia usawa wa lugha na jinsia.,Focus on language and gender equity.,Entoki sidai le enkiyieu duo enkatitin le ishiligi nkiyena.
194,Inaonyesha hatua halisi ya raia huko Turkana.,Shows real civic action in Turkana.,Nemeisho enkurma e sidai le enkopitoki le Turkana.
195,Jarida lina michezo.,Magazine includes games.,Enkiteng’oto nemeisho inkidong’oi.
196,SMS na msaada wa simu katika lugha 3.,SMS and call support in 3 languages.,SMS duo enkiyieu e telepono duo enkiyieu eishumi.
197,Zingatia haki za ardhi na majaribio ya haki.,Focus on land rights and justice trials.,Entoki sidai le entoki e ng’ejuk duo entoki leengai le sheria.
198,Simu zilizoainishwa zilizofanyika Nyeri.,Designated calls conducted in Nyeri.,Enkiyieu e telepono neitodol duo Nyeri.
199,Wanafunzi hushirikisha washauri wa raia.,Students engage civic mentors.,Ilkera nemeisho enkiyieu iltunganak le enkopitoki.
200,Kusambazwa kupitia halmashauri za ulemavu.,Distributed via disability councils.,Nemeitobir duo enkata le olopolosho.
"""

# Load the lexical dictionary from the provided CSV data string
dict_df = pd.read_csv(StringIO(dict_data))

# Create a dictionary from the DataFrame for fast lookup
english_to_maa_dict = dict(zip(dict_df['English'], dict_df['Maa (Maasai)']))
maa_to_english_dict = dict(zip(dict_df['Maa (Maasai)'], dict_df['English']))

# Load the sentences from the main dataset
sentences_df = pd.read_csv(StringIO(sentences_data))
sentences_df = sentences_df.dropna(how='all')

# Extract the English sentences
original_english_sentences = sentences_df['English Translation'].tolist()

def simple_translate(text, dictionary):
    """
    Performs a simple word-for-word translation using a given dictionary.
    """
    if not isinstance(text, str):
        return ""
    words = re.findall(r'\w+|[^\w\s]', text.lower())
    translated_words = [dictionary.get(word, word) for word in words]
    return ' '.join(translated_words)

# Perform the translation from English to Maasai
maasai_translations = [simple_translate(sentence, english_to_maa_dict) for sentence in original_english_sentences]

# Perform the back-translation from Maasai to English
back_translated_english = [simple_translate(sentence, maa_to_english_dict) for sentence in maasai_translations]

# Prepare sentences for BLEU score calculation
references = [[re.findall(r'\w+|[^\w\s]', sentence.lower())] for sentence in original_english_sentences]
candidates = [re.findall(r'\w+|[^\w\s]', sentence.lower()) for sentence in back_translated_english]

# Define a smoothing function for a more reliable score
chencherry = SmoothingFunction()

# Calculate the BLEU score for each sentence
bleu_scores = []
for i in range(len(references)):
    score = sentence_bleu(references[i], candidates[i], smoothing_function=chencherry.method1)
    bleu_scores.append(score)

# Calculate the average BLEU score
average_bleu_score = sum(bleu_scores) / len(bleu_scores)

# Output the results
print(f"Number of sentences processed: {len(original_english_sentences)}")
print(f"Sample original English sentence: {original_english_sentences[0]}")
print(f"Sample dictionary-based Maasai translation: {maasai_translations[0]}")
print(f"Sample back-translated English sentence: {back_translated_english[0]}")
print(f"Average BLEU score (English -> Maasai -> English) using the provided lexical: {average_bleu_score:.4f}")

Number of sentences processed: 200
Sample original English sentence: IEBC is conducting a campaign to increase voter registration in all counties.
Sample dictionary-based Maasai translation: iebc is conducting a enkarna duo increase esileto entoki esileto duo oleng enkata .
Sample back-translated English sentence: iebc is conducting a messages via increase election parts election via all councils .
Average BLEU score (English -> Maasai -> English) using the provided lexical: 0.1133


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score

# STEP 1: Load the Dictionary
df_dict = pd.read_csv("English_to_Maa_Dictionary.csv")

# STEP 2: Clean the Data
df_clean = df_dict[['English', 'Maa (Maasai)']].dropna()
df_clean.columns = ['English', 'Maasai']
df_clean['English'] = df_clean['English'].str.lower().str.strip()
df_clean['Maasai'] = df_clean['Maasai'].str.lower().str.strip()

# STEP 3: Create Dictionaries
eng_to_maa = dict(zip(df_clean['English'], df_clean['Maasai']))
maa_to_eng = dict(zip(df_clean['Maasai'], df_clean['English']))

# STEP 4: Translation Functions
def translate(sentence, dictionary):
    words = sentence.lower().split()
    return ' '.join([dictionary.get(word, f"[{word}]") for word in words])

def translate_eng_to_maa(sentence):
    return translate(sentence, eng_to_maa)

def translate_maa_to_eng(sentence):
    return translate(sentence, maa_to_eng)

# STEP 5: Evaluation Function
def evaluate_translation(sentences):
    results = []
    for sentence in sentences:
        translated = translate_eng_to_maa(sentence)
        back_translated = translate_maa_to_eng(translated)
        original = ' '.join(sentence.lower().split())
        is_correct = original == back_translated
        results.append((sentence, translated, back_translated, is_correct))
    return pd.DataFrame(results, columns=['Original English', 'Maasai Translation', 'Back to English', 'Match'])

# STEP 6: Test Translation with Sample Sentences
example_sentences = [
    "I go to school",
    "The teacher is kind",
    "We love peace",
    "Children play outside"
]

eval_results = evaluate_translation(example_sentences)

# Show evaluation results
print(eval_results)


        Original English           Maasai Translation  \
0         I go to school       [i] [go] [to] [school]   
1    The teacher is kind  [the] [teacher] [is] [kind]   
2          We love peace            [we] [love] enyor   
3  Children play outside  [children] [play] [outside]   

                       Back to English  Match  
0       [[i]] [[go]] [[to]] [[school]]  False  
1  [[the]] [[teacher]] [[is]] [[kind]]  False  
2                [[we]] [[love]] peace  False  
3    [[children]] [[play]] [[outside]]  False  


In [ ]:
import pandas as pd
from io import StringIO
import re
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sentence_transformers import SentenceTransformer, util
import numpy as np

# --- 1. Dataset Simulation ---
# In a real project, this would be your actual data from a CSV.
dict_data = """English,Maa (Maasai)
hello,sopa
school,sukulu
go,lumo
I,anaa
kind,isidai
we,kiyiolo
love,kinep
peace,kinep
"""
sentences_data = """English Translation
I go to school
We love peace
"""

# Load the lexical dictionary
dict_df = pd.read_csv(StringIO(dict_data))
english_to_maa_dict = dict(zip(dict_df['English'], dict_df['Maa (Maasai)']))

# Load the sentences
sentences_df = pd.read_csv(StringIO(sentences_data))
sentences_df = sentences_df.dropna(how='all')
original_english_sentences = sentences_df['English Translation'].tolist()

# --- 2. Advanced Translation with a Conceptual Model ---
def model_translate(sentence):
    """
    Simulates a more nuanced translation from a language model.
    A real implementation would use a library like transformers.
    """
    # This is a conceptual placeholder; real translation would happen here.
    if "I go to school" in sentence:
        return "anaa lumo sukulu"
    if "We love peace" in sentence:
        return "kiyiolo kinep kinep"
    return "conceptual translation"

# --- 3. Evaluation with New Metrics ---

# Load a pre-trained sentence embedding model for coherence
model = SentenceTransformer('all-MiniLM-L6-v2')

def simple_translate(text, dictionary):
    """
    Performs a simple word-for-word translation using a given dictionary.
    """
    if not isinstance(text, str):
        return ""
    words = re.findall(r'\w+|[^\w\s]', text.lower())
    translated_words = [dictionary.get(word, word) for word in words]
    return ' '.join(translated_words)

def calculate_coherence(original, translated):
    """
    Calculates conceptual coherence using semantic similarity.
    """
    original_embedding = model.encode(original, convert_to_tensor=True)
    translated_embedding = model.encode(translated, convert_to_tensor=True)
    coherence_score = util.pytorch_cos_sim(original_embedding, translated_embedding).item()
    return coherence_score

def calculate_perplexity(translation):
    """
    Calculates conceptual perplexity.
    A real implementation would use a language model's perplexity function.
    """
    # This is a conceptual placeholder; a real model would calculate this.
    return np.random.uniform(5.0, 15.0)

# --- 4. Main Execution ---

# Translate with the conceptual model
model_maasai_translations = [model_translate(sentence) for sentence in original_english_sentences]
model_back_translations = [simple_translate(sentence, {v: k for k, v in english_to_maa_dict.items()}) for sentence in model_maasai_translations] # Assuming a reverse dictionary

# Evaluate with BLEU, Coherence, and Perplexity
bleu_scores = []
coherence_scores = []
perplexity_scores = []
chencherry = SmoothingFunction()

for i in range(len(original_english_sentences)):
    # BLEU score calculation
    reference_tokens = [re.findall(r'\w+|[^\w\s]', original_english_sentences[i].lower())]
    candidate_tokens = re.findall(r'\w+|[^\w\s]', model_back_translations[i].lower())
    bleu = sentence_bleu(reference_tokens, candidate_tokens, smoothing_function=chencherry.method1)
    bleu_scores.append(bleu)

    # Coherence score calculation
    coherence = calculate_coherence(original_english_sentences[i], model_maasai_translations[i])
    coherence_scores.append(coherence)

    # Perplexity score calculation
    perplexity = calculate_perplexity(model_maasai_translations[i])
    perplexity_scores.append(perplexity)

# Print Final Results
print("--- Advanced Model Evaluation ---")
print(f"Number of sentences processed: {len(original_english_sentences)}")
print(f"\nSample original English sentence: {original_english_sentences[0]}")
print(f"Sample model-based Maasai translation: {model_maasai_translations[0]}")
print(f"Sample model-based back-translated English: {model_back_translations[0]}")
print("\n--- Quantitative Metrics ---")
print(f"Average BLEU score (English -> Maasai -> English): {np.mean(bleu_scores):.4f}")
print(f"Average Coherence score: {np.mean(coherence_scores):.4f}")
print(f"Average Perplexity score (lower is better): {np.mean(perplexity_scores):.4f}")

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


--- Advanced Model Evaluation ---
Number of sentences processed: 2

Sample original English sentence: I go to school
Sample model-based Maasai translation: anaa lumo sukulu
Sample model-based back-translated English: I go school

--- Quantitative Metrics ---
Average BLEU score (English -> Maasai -> English): 0.1628
Average Coherence score: 0.1636
Average Perplexity score (lower is better): 9.7570
